# **Data** **Wrangling**

* Clean data

* Merge and concatenate data

## Dataset 1
<a id='1'></a>
###### https://github.com/UCSM-DUE/IWG_hatespeech_public


In [1]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving german hatespeech refugees.csv to german hatespeech refugees.csv


In [2]:
import io
df1 = pd.read_csv(io.BytesIO(uploaded['german hatespeech refugees.csv']))

In [3]:
df1.head()

,Tweet,HatespeechOrNot (Expert 1),HatespeechOrNot (Expert 2),Hatespeech Rating (Expert 2)
0,#Rosenmontag ist abgesagt. #Rapefugees also wi...,YES,NO,5
1,bitte nicht die #Türkei zum #EU-Mitglied mache...,NO,NO,1
2,Wieso bekommen #rapefugees mehr als unsere Har...,NO,NO,3
3,Den verfluchten #Rapefugees den Krieg erklären...,NO,YES,4
4,War das Wochenende im Ruhrpott unterwegs. Über...,YES,YES,6


## Relabel data

In [4]:
df1['HateSpeech'] = (df1['Hatespeech Rating (Expert 2)'] >= 3)
df1.replace({True:1,False:0},inplace=True)

In [8]:
df1.drop(['HatespeechOrNot (Expert 1)', 'HatespeechOrNot (Expert 2)', 'Hatespeech Rating (Expert 2)'], axis=1, inplace=True)

In [9]:
df1.rename(columns={'Tweet': 'message'}, inplace=True)

In [10]:
df1.head()

,message,HateSpeech
0,#Rosenmontag ist abgesagt. #Rapefugees also wi...,1
1,bitte nicht die #Türkei zum #EU-Mitglied mache...,0
2,Wieso bekommen #rapefugees mehr als unsere Har...,1
3,Den verfluchten #Rapefugees den Krieg erklären...,1
4,War das Wochenende im Ruhrpott unterwegs. Über...,1


In [11]:
df1['HateSpeech'].value_counts()

0    283
1    186
Name: HateSpeech, dtype: int64

In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469 entries, 0 to 468
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message     469 non-null    object
 1   HateSpeech  469 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.5+ KB


# Datset 2
<a id='2'></a>

#####  2017 - Offending Statements / Hate Speech towards Foreigners (http://ub-web.de/research/index.html)

In [13]:
uploaded = files.upload()

Saving comments.csv to comments.csv


In [14]:
df2 = pd.read_csv(io.BytesIO(uploaded['comments.csv']))

In [15]:
df2.head()

,comment_id,post_id,anonymized_user,message,created_at
0,1,1,d766dfb93914e8da,gleich an die wand stellen und erschiessen..,2015-03-28 01:59:28
1,2,1,6e49f84e2a5a1f05,nicht dass ich der Grundbotschaft dieses Post'...,2015-03-28 01:59:28
2,3,1,6ab1ef4615c45a79,"Das mit dem ""an die Wand stellen und erschiessen""",2015-03-28 01:59:28
3,4,1,6ab1ef4615c45a79,"Seit dem ""an die Wand stellen und erschiessen""...",2015-03-28 01:59:28
4,5,1,03c34eadeee054d7,Ja ja die Kriminelle Heimatpartei FPÖ von Kind...,2015-03-28 01:59:28


In [18]:
uploaded = files.upload()

Saving annotated_comments.csv to annotated_comments.csv


In [19]:
df3 = pd.read_csv(io.BytesIO(uploaded['annotated_comments.csv']))

In [20]:
df3.head(2)

,comment_id,annotator_id,entry_number,valence,target_type
0,1,1,1,2,2
1,1,3,1,2,2


In [21]:
#several annotations for each comment, catch the maximum
df3 = df3.groupby('comment_id').max().reset_index()

In [22]:
# Merge datasets by comment_id, annotated comment is hate speech, other comments are neutral
df_merged = df2.merge(df3, how='left', on='comment_id')

In [24]:
df_merged.drop(['comment_id', 'post_id', 'anonymized_user','created_at','annotator_id', 'entry_number'], axis=1, inplace=True)

In [25]:
df_merged.head(2)

,message,valence,target_type
0,gleich an die wand stellen und erschiessen..,2.0,2.0
1,nicht dass ich der Grundbotschaft dieses Post'...,NaN,NaN


## Relabel data

In [26]:
#Hate Speech when target_type != 0

df_merged['target_type'] = df_merged['target_type'].fillna(0)
df_merged['HateSpeech'] = (df_merged['target_type'] != 0)
df_merged.replace({True:1,False:0},inplace=True)

In [28]:
df_merged.drop(['valence', 'target_type'], axis=1, inplace=True)

In [29]:
df_merged.head(3)

,message,HateSpeech
0,gleich an die wand stellen und erschiessen..,1
1,nicht dass ich der Grundbotschaft dieses Post'...,0
2,"Das mit dem ""an die Wand stellen und erschiessen""",0


In [30]:
df_merged['HateSpeech'].value_counts()

0    5162
1     674
Name: HateSpeech, dtype: int64

In [31]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5836 entries, 0 to 5835
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message     5836 non-null   object
 1   HateSpeech  5836 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 136.8+ KB



# Dataset 3
<a id='3'></a>
###### https://github.com/uds-lsv/GermEval-2018-Data

In [32]:
uploaded = files.upload()

Saving germeval2018.training.csv to germeval2018.training.csv


In [33]:
df4 = pd.read_csv(io.BytesIO(uploaded['germeval2018.training.csv']),sep=";", header=0)

In [34]:
df4.head()

,tweet,anno,anno2
0,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER
1,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER
2,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER
3,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT
4,"@Dirki_M Ja, aber wo widersprechen die Zahlen ...",OTHER,OTHER


In [35]:
uploaded = files.upload()

Saving germeval2018.test.csv to germeval2018.test.csv


In [36]:
df5 = pd.read_csv(io.BytesIO(uploaded['germeval2018.test.csv']),sep=";")

In [37]:
df5.head()

,tweet,anno,anno2
0,@Tom174_ @davidbest95 Meine Reaktion; |LBR| Ni...,OTHER,OTHER
1,"#Merkel rollt dem Emir von #Katar, der islamis...",OTHER,OTHER
2,„Merle ist kein junges unschuldiges Mädchen“ K...,OTHER,OTHER
3,@umweltundaktiv Asylantenflut bringt eben nur ...,OFFENSE,ABUSE
4,@_StultaMundi Die Bibel enthält ebenfalls Gese...,OTHER,OTHER


In [39]:
#merge train and test dataset
df_germ = pd.concat([df4, df5], ignore_index=True, sort=False)

In [40]:
df_germ.head()

,tweet,anno,anno2
0,@Martin28a Sie haben ja auch Recht. Unser Twee...,OTHER,OTHER
1,@ahrens_theo fröhlicher gruß aus der schönsten...,OTHER,OTHER
2,@dushanwegner Amis hätten alles und jeden gewä...,OTHER,OTHER
3,@spdde kein verläßlicher Verhandlungspartner. ...,OFFENSE,INSULT
4,"@Dirki_M Ja, aber wo widersprechen die Zahlen ...",OTHER,OTHER


In [43]:
#clean data
df_germ['tweet'] = df_germ['tweet'].replace("@[A-Za-z0-9]+","", regex=True)

## Relabel

In [45]:
#No Hate Speech when annotated 'OTHER'

df_germ['HateSpeech'] = (df_germ['anno2'] == 'OTHER')

df_new.replace({True:0,False:1},inplace=True)

In [46]:
df_germ.drop(['anno', 'anno2'], axis=1, inplace=True)

In [47]:
df_germ.rename(columns={'tweet': 'message'}, inplace=True)

In [48]:
df_germ.head()

,message,HateSpeech
0,@Martin28a Sie haben ja auch Recht. Unser Twee...,True
1,@ahrens_theo fröhlicher gruß aus der schönsten...,True
2,@dushanwegner Amis hätten alles und jeden gewä...,True
3,@spdde kein verläßlicher Verhandlungspartner. ...,False
4,"@Dirki_M Ja, aber wo widersprechen die Zahlen ...",True


In [49]:
df_germ['HateSpeech'].value_counts()

True     5567
False    2838
Name: HateSpeech, dtype: int64

# Dataset 4
<a id='4'></a>
###### https://hasocfire.github.io/hasoc/2019/dataset.html

In [50]:
uploaded = files.upload()

Saving german_dataset_2019.csv to german_dataset_2019.csv


In [51]:
df6 = pd.read_csv(io.BytesIO(uploaded['german_dataset_2019.csv']),sep=";")

In [52]:
df6.head(3)

,text_id,text,task_1,task_2
0,hasoc_de_1,Frank Rennicke – Ich bin stolz https://t.co/Cm...,NOT,NONE
1,hasoc_de_2,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,NOT,NONE
2,hasoc_de_3,#Koeln Mohamed erkennt kein deutsches Recht so...,NOT,NONE


In [53]:
uploaded = files.upload()

Saving hasoc_de_test_gold_2019.csv to hasoc_de_test_gold_2019.csv


In [54]:
df7 = pd.read_csv(io.BytesIO(uploaded['hasoc_de_test_gold_2019.csv']),sep=";")

In [55]:
df7.head(3)

,text_id,text,task_1,task_2
0,hasoc_de_5270,"Die Grüne Intelligenz Bestie 'Eckhardt', Prang...",HOF,HATE
1,hasoc_de_5271,"Das Deutschlandlied, Sinn und Unsinn https:/...",NOT,NONE
2,hasoc_de_5272,#Eurovision https://t.co/24T8fzUI44,NOT,NONE


In [56]:
#merge dataset
df_hasoc = pd.concat([df6, df7], ignore_index=True, sort=False)

In [57]:
df_hasoc.head()

,text_id,text,task_1,task_2
0,hasoc_de_1,Frank Rennicke – Ich bin stolz https://t.co/Cm...,NOT,NONE
1,hasoc_de_2,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,NOT,NONE
2,hasoc_de_3,#Koeln Mohamed erkennt kein deutsches Recht so...,NOT,NONE
3,hasoc_de_4,#SaudiArabien ist eine brutale islamische Dikt...,NOT,NONE
4,hasoc_de_5,Bundespolizei #München hat im 1. Quartal 2019 ...,NOT,NONE


## Relabel

In [58]:
#No Hate Speech when annotated 'NOT/NONE'

df_hasoc['HateSpeech'] = (df_hasoc['task_1'] == 'NOT')

df_hasoc.replace({True:0,False:1},inplace=True)

In [59]:
df_hasoc.drop(['task_1', 'task_2','text_id'], axis=1, inplace=True)

In [60]:
df_hasoc.rename(columns={'text': 'message'}, inplace=True)

In [61]:
df_hasoc.head()

,message,HateSpeech
0,Frank Rennicke – Ich bin stolz https://t.co/Cm...,0
1,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,0
2,#Koeln Mohamed erkennt kein deutsches Recht so...,0
3,#SaudiArabien ist eine brutale islamische Dikt...,0
4,Bundespolizei #München hat im 1. Quartal 2019 ...,0


In [62]:
df_hasoc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4669 entries, 0 to 4668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message     4669 non-null   object
 1   HateSpeech  4669 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 73.1+ KB


In [63]:
df_hasoc['HateSpeech'].value_counts()

0    4126
1     543
Name: HateSpeech, dtype: int64

# Concatenate all datasets
<a id='concat'></a>

In [66]:
#merge datasets

df_hatespeech = pd.concat([df1, df_merged, df_germ, df_hasoc], ignore_index=True, sort=False)

In [69]:
#clean data
df_hatespeech['message'] = df_hatespeech['message'].replace("@[A-Za-z0-9]+","", regex=True)
df_hatespeech.head()

,message,HateSpeech
0,#Rosenmontag ist abgesagt. #Rapefugees also wi...,1
1,bitte nicht die #Türkei zum #EU-Mitglied mache...,0
2,Wieso bekommen #rapefugees mehr als unsere Har...,1
3,Den verfluchten #Rapefugees den Krieg erklären...,1
4,War das Wochenende im Ruhrpott unterwegs. Über...,1


In [70]:
df_hatespeech.shape

(19379, 2)

In [71]:
df_hatespeech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19379 entries, 0 to 19378
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message     19379 non-null  object
 1   HateSpeech  19379 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 302.9+ KB


In [ ]:
#Export data
from google.colab import files
df_hate_speech.to_csv('hatespeech.csv') 
files.download('hatespeech.csv')